<a href="https://colab.research.google.com/github/jacquesbilombe/Plustrategy/blob/main/datafetcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For more information on the API, check this [link](https://pypi.org/project/sportradar/).

In [1]:
import requests

from bs4 import BeautifulSoup

In [2]:
cookies = {
    '_gcl_au': '1.1.1494853104.1715364312',
    '_ga': 'GA1.1.1918003453.1715364313',
    '__qca': 'P0-1691788869-1715364322655',
    'FCNEC': '%5B%5B%22AKsRol9XuTvtcA-KtSeor_fgzdQW58RK_BpfZB7Sdj5I2XedsLxKLTo20tcRe8s8L53_TyoQtuRWeKKXFto4oIVAygDrfxFElND8Xx-JTSyxHbl3W_lqOhjJ7snWZ73Xr-3PupQZsWVMa3yJ19Vv92Tc6i2muqe-rg%3D%3D%22%5D%5D',
    '__gads': 'ID=b5947f16edb0403d:T=1715364320:RT=1715366201:S=ALNI_MaJIyMoeqZaO--UX-FZHBggRDNdJA',
    '__gpi': 'UID=00000a257e3bc349:T=1715364320:RT=1715366201:S=ALNI_MbCel5XYfhe-MDCNwKoU5oysKgFOA',
    '__eoi': 'ID=934a22df78350c96:T=1715364320:RT=1715366201:S=AA-AfjZuN6AaSh_XAQiV6aYPqXtz',
    '_ga_3KF4XTPHC4': 'GS1.1.1715364313.1.1.1715366296.60.0.0',
    '_ga_QH2YGS7BB4': 'GS1.1.1715364313.1.1.1715366296.0.0.0',
    '_ga_HNQ9P9MGZR': 'GS1.1.1715364319.1.1.1715366296.57.0.0',
}

headers = {
    'accept': '*/*',
    'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7,pt-BR;q=0.6,pt;q=0.5,fr-CD;q=0.4,ja-JP;q=0.3,ja;q=0.2',
    'cache-control': 'max-age=0',
    # 'cookie': '_gcl_au=1.1.1494853104.1715364312; _ga=GA1.1.1918003453.1715364313; __qca=P0-1691788869-1715364322655; FCNEC=%5B%5B%22AKsRol9XuTvtcA-KtSeor_fgzdQW58RK_BpfZB7Sdj5I2XedsLxKLTo20tcRe8s8L53_TyoQtuRWeKKXFto4oIVAygDrfxFElND8Xx-JTSyxHbl3W_lqOhjJ7snWZ73Xr-3PupQZsWVMa3yJ19Vv92Tc6i2muqe-rg%3D%3D%22%5D%5D; __gads=ID=b5947f16edb0403d:T=1715364320:RT=1715366201:S=ALNI_MaJIyMoeqZaO--UX-FZHBggRDNdJA; __gpi=UID=00000a257e3bc349:T=1715364320:RT=1715366201:S=ALNI_MbCel5XYfhe-MDCNwKoU5oysKgFOA; __eoi=ID=934a22df78350c96:T=1715364320:RT=1715366201:S=AA-AfjZuN6AaSh_XAQiV6aYPqXtz; _ga_3KF4XTPHC4=GS1.1.1715364313.1.1.1715366296.60.0.0; _ga_QH2YGS7BB4=GS1.1.1715364313.1.1.1715366296.0.0.0; _ga_HNQ9P9MGZR=GS1.1.1715364319.1.1.1715366296.57.0.0',
    'dnt': '1',
    'if-none-match': 'W/"f9dc1f6f67"',
    'priority': 'u=1, i',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'x-requested-with': '77b62c',
}

response = requests.get(
    'https://www.sofascore.com/api/v1/sport/football/scheduled-events/2024-05-10',
    cookies=cookies,
    headers=headers,
)

In [3]:
response

<Response [200]>

In [4]:
# Now we can get the data we want
today_games = response.json()
today_games

{'events': [{'tournament': {'name': 'Premier League',
    'slug': 'premier-league',
    'category': {'name': 'England',
     'slug': 'england',
     'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
     'id': 1,
     'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England'},
     'flag': 'england',
     'alpha2': 'EN'},
    'uniqueTournament': {'name': 'Premier League',
     'slug': 'premier-league',
     'category': {'name': 'England',
      'slug': 'england',
      'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
      'id': 1,
      'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England'},
      'flag': 'england',
      'alpha2': 'EN'},
     'userCount': 1688054,
     'id': 17,
     'hasEventPlayerStatistics': True,
     'displayInverseHomeAwayTeams': False},
    'priority': 600,
    'id': 1},
   'season': {'name': 'Premier League 23/24',
    'year': '23/24',
    'editor': False,
    'id': 52186},
   'roundInfo': {'round': 37},
   'customId': 'rsT'

In [ ]:
"""odds = requests.get('https://www.sofascore.com/api/v1/sport/football/odds/1/2024-05-10', cookies=cookies, headers=headers)

# Now we can get the data we want
today_odds = odds.json()
today_odds"""

In [7]:
matches = today_games.values()
type(matches)

dict_values